# Reddit Scraping
In order to scrape posts from Reddit, we used the [`praw` python library](https://github.com/praw-dev/praw). 
In order to use that, we need:
- a Reddit account.
- a Reddit app ([here](https://www.reddit.com/prefs/apps) you can create one). Here you'll find the client id under the app name once you create it, as well as the client secret and the user agent, which is your app name.

Once we have those, we can access reddit from the terminal using these steps(it is also possible to access as a user by adding username and account if you want to use it in write mode):

In [1]:
#pip install praw
import praw
reddit = praw.Reddit(client_id='xSEBvh4afPIXiJ5RE07K0g', client_secret='iTMyTT4CR8gDckcwbzy2ZyoqxQgwiA', user_agent='scrape_analysis')

We can analyse posts, post requirements etc. for a single subreddit or for all of them(by using the subreddit 'all'); we can also select top posts or do searches. 
# sistemare data utc

In [15]:
for i in reddit.subreddit('all').top(limit=5): #this will print the top posts all time
    print(i.title + ' Author: ' + i.author.name +  ' Link: https://www.reddit.com' + i.permalink + ' Subreddit: r/' + i.subreddit.display_name + ' Upvote Ratio: ' + str(i.upvote_ratio) + ' Date: '+str(i.created_utc))


Humans ... Author: beautifulplanet Link: https://www.reddit.com/r/funny/comments/skap0y/humans/ Subreddit: r/funny Upvote Ratio: 0.82 Date: 1643972642.0
A newlywed couple was enjoying their first dance when suddenly someone unexpectedly joined them. Author: Arl107 Link: https://www.reddit.com/r/MadeMeSmile/comments/skaiwx/a_newlywed_couple_was_enjoying_their_first_dance/ Subreddit: r/MadeMeSmile Upvote Ratio: 0.96 Date: 1643972046.0
These calls don’t ever end like that for me Author: daaaangdood Link: https://www.reddit.com/r/Unexpected/comments/sk8x76/these_calls_dont_ever_end_like_that_for_me/ Subreddit: r/Unexpected Upvote Ratio: 0.94 Date: 1643965794.0
Falling into the abyss Author: YoutubeMaikEyy Link: https://www.reddit.com/r/memes/comments/ska4m5/falling_into_the_abyss/ Subreddit: r/memes Upvote Ratio: 0.94 Date: 1643970517.0
Murica really gunning for least educated Author: FlameOfGod Link: https://www.reddit.com/r/facepalm/comments/skamo7/murica_really_gunning_for_least_educate